In [ ]:
import pandas as pd
import fastf1 as f1 



# This is what I am expecting the result to contain (Doesn't have to be in the same order) 
""" drivers_2022_list = ["Zhou Guanyu", "Valtteri Bottas", "Pierre Gasly", "Yuki Tsunoda","Fernando Alonso", "Esteban Ocon","Sesbastian Vettel","Lance Stroll","Charles Leclerc","Carlos Sainz Jr.","Kevin Magnussen","Mick Schumacher","Daniel Ricciardo", "Lando Norris","Lewis Hamilton","George Russell"," Max Verstappen"," Sergio Perez", "Nicholas Latifi", "Alexander Albon"] """ 


# I am choosing the Australian GP randomly because it contains all the main drivers in their main team.  
F1_2022_AUS = f1.get_session(2022, "Australian Grand Prix", "R" )
F1_2022_AUS.load(telemetry=True, laps=True)
drivers_2022_list = F1_2022_AUS.results['FullName'].tolist()

F1_2023_AUS = f1.get_session(2023, 'Australian Grand Prix', 'R')
F1_2023_AUS.load()
drivers_2023_list = F1_2023_AUS.results['FullName'].tolist()
drivers_2023_list.append('Daniel Ricciardo')

# we are gonna get the list of races for each year as well as 2024 
# we are gonna load in the list of name into the df 

# schedule for 2022 season 
schedule22 = f1.get_event_schedule(2022, include_testing=False)
schedule_2022 = schedule22[ 'EventName'].tolist()

# schedule for 2023 season 
schedule23 = f1.get_event_schedule(2023, include_testing=False)
schedule_2023 = schedule23['EventName'].tolist()

# schedule for 2024 season 
schedule24 = f1.get_event_schedule(2024, include_testing=False)
schedule_2024 = schedule24['EventName'].tolist()


# each of the driver names will be rows and then columns will be GP_Year_Session_Tyre-type 
# any area in which we don't have data for like maybe that driver didn't participate in that GP or they didn't use that tyre compound we will use n/a

for driver in drivers_2022_list:
    for GP in schedule_2022:
        
        get_data(2022, GP , driver)
    

#get columns for data 
session_list_2022 = ['FP1', 'FP2','FP3','Q']
tyre_list = ['Soft','Medium', 'Hard']
columns = []

for GP in schedule22:
  for session in session_list_2022:
        for tyre in tyre_list:
           if GP == 'Emilia Romagna Grand Prix' or  GP == 'Austrian Grand Prix' or GP == 'Sao Paulo Grand Prix': 
                if session == 'FP3': continue
           else:
            columns.append(GP+'_'+2022+'_'+ session +'_'+tyre) 


session_list_2023 = ['FP1', 'FP2','FP3','Q' ,' SQ']
tyre_list = ['Soft','Medium', 'Hard']
for GP in schedule23:
  for session in session_list_2023:
        for tyre in tyre_list:
           if GP == 'Azerbaijan Grand Prix' or  GP == 'Austrian Grand Prix' or GP == 'Sao Paulo Grand Prix' or GP =='Belgian Grand Prix' or GP == 'Qatar Grand Prix' or GP == ' United States Grand Prix': 
                if session == 'FP3': continue
                if session == 'SQ': columns.append(GP+'_'+2022+'_'+ session +'_'+tyre) 
           else:
            if session == 'SQ': continue 
            columns.append(GP+'_'+2022+'_'+ session +'_'+tyre) 


# we are trying to get tyre type and then use this format " GP_Year_Session_Tyre-type " to input them as columns for each drivers 
# look out for "LapTime" and "IsPersonalBest" and "Coumpound" and "FastF1Generated" and "IsAccurate"
# for each GP we will want 3 type of data (fp1 fp2 fp3) then each of that soft, medium, hard 
# sprint race we only get fp1, qualifying session and fp2 for 2022 
# sprint race for 2023 we need fp1 , Qualifying (Q1,Q2,Q3), sprint Qualifying (1,2,3 new mediums for 1 and 2 and soft for 3) 

# we are gonna get data for both practice and qualifying session. 

# we are gonna try to return a 2D array.and they will serve as a parallel array. One will have the title "GP_Year_Session_Tyre-type" then the other will have the time and if not it will be N/A  
# columns will be driver names 
def get_data(year ,GP, Driver):
    data = []
    session_list = ['FP1', 'FP2','FP3','Q']
    tyre_list = ['Soft','Medium', 'Hard']
    for index in session_list:
        session = f1.get_session(year, GP, index)
        session.load() 
        for tyre in tyre_list:
            session.laps.pick_drivers(Driver).pick_fastest()['Compound']       
            data[Driver][GP+'_'+year+'_'+index+'_'+session.laps.pick_drivers(Driver).pick_fastest()['Compound']] = session.laps.pick_drivers(Driver).pick_fastest()
    return data 

def sprint_data(year, GP, Driver):
        data = []
        session_list = ['FP1', 'FP2','Q']
        for index in session_list:
            session = f1.get_session(year, GP, index)
            session.load() 
            session.laps.pick_drivers(Driver).pick_fastest()['Compound']       
            data[Driver][GP+'_'+year+'_'+index+'_'+session.laps.pick_drivers(Driver).pick_fastest()['Compound']] = session.laps.pick_drivers(Driver).pick_fastest()
        return data 



    

IndentationError: expected an indented block after 'for' statement on line 46 (2668533335.py, line 59)

In [ ]:


def get_data(year ,GP, Driver):
    data = []
    session_list = ['FP1', 'FP2','FP3','Q']
    tyre_list = ['Soft','Medium', 'Hard']
    
    for index in session_list:
        session = f1.get_session(year, GP, index)
        session.load() 
        for tyre in tyre_list:
            time = session.laps.pick_drivers(Driver).pick_compounds(tyre).pick_fastest()  
            try:
                """ data[Driver][GP+'_'+year+'_'+index+'_'+tyre] = time['LapTime'] """
                data.append(time['LapTime'])
            except:
                data.append('N/A')
            
    return data 
# storing time delta object 
arr = get_data(2022, 1, 'VER')
for row in arr:
    for col in row:
        print(col+':'+)

# we want the index or row to be the driver names 
# we could get all the data for 1 driver at time, put that into an array just in a line 
# then we package it as 2D array. 
# we will also keep track of the column names in another array. 
# at the end we will put all of that into a dataframe 


In [9]:
import pandas as pd
import fastf1 as f1 
from timple.timedelta import strftimedelta as td

#["GP, year, session, tyre"]
F1_2022_AUS = f1.get_session(2022, "Australian Grand Prix", "R" )
F1_2022_AUS.load(telemetry=True, laps=True)
drivers_2022_list = F1_2022_AUS.results['Abbreviation'].tolist()

F1_2023_AUS = f1.get_session(2023, 'Australian Grand Prix', 'R')
F1_2023_AUS.load()
drivers_2023_list = F1_2023_AUS.results['Abbreviation'].tolist()
drivers_2023_list.append('RIC')


drivers_list_combined = list(set(drivers_2022_list+drivers_2023_list))
# this is how we will get the fastest time for each 
session = f1.get_session(2022, 'Bahrain Grand Prix','FP1')
session.load(laps=True, telemetry=True, weather=True, messages=False)
datas =[]
print(session.weather_data) 
def get_weather(session):
    session_weather = session.weather_data
    rainCheck = session_weather.loc[session_weather['Rainfall'] == True] 
    print(rainCheck)
    if rainCheck.empty:
        return False
    else:
        return True
    

weather = get_weather(session)
if weather is not True:
     print( weather )
print(weather)

time = session.laps.pick_drivers('HAM').pick_compounds('Hard').pick_fastest()['LapTime']
datas.append(td(time, '%m:%s.%ms'))
print(datas)
data=[]
dataset = {'Name': drivers_list_combined}
pf = pd.DataFrame(dataset)
print(pf)
index = 'Austrian Grand Prix_2022_SQ_Medium'
info = index.rsplit("_")
session1 = f1.get_session(int(info[1]), info[0], info[2])
session1.load(laps=True, telemetry=True, weather=False, messages=False)  
for i in drivers_list_combined:
    time1 = session1.laps.pick_drivers(i).pick_compounds(info[3]).pick_fastest()  
    print(time1)
    print()
    try:
            print(i)
            data.append(time1['LapTime'])
    except:
            data.append('N/A')

pf[index] = data
print(pf)



""" df = pd.DataFrame(data)
print(df)  """

""" fastest = session.laps.pick_drivers('VER').pick_compounds('Medium').pick_fastest()['LapTime']
try:
   
    print(fastest) # this is a timedelta object 
except:
    print('None')   """  
       

""" # initialize list of lists
data = [['tom', 10], ['nick', 15], ['juli', 14]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['Name', 'Age'])

print(df) """



core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 16 completed the race distance 00:00.140000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['16', '11', '63', '44', '4', '3', '31', '77', '10', '23', '24',

                     Time  AirTemp  Humidity  Pressure  Rainfall  TrackTemp  \
0  0 days 00:00:54.194000     20.1      36.0    1016.2     False       36.9   
1  0 days 00:01:54.189000     20.2      36.0    1016.2     False       37.0   
2  0 days 00:02:54.194000     20.0      37.0    1016.2     False       36.7   
3  0 days 00:03:54.186000     19.9      37.0    1016.0     False       36.7   
4  0 days 00:04:54.189000     19.9      37.0    1016.2     False       36.6   
..                    ...      ...       ...       ...       ...        ...   
77 0 days 01:17:54.442000     19.6      36.0    1016.0     False       32.0   
78 0 days 01:18:54.440000     19.7      36.0    1016.0     False       32.2   
79 0 days 01:19:54.441000     19.6      36.0    1016.0     False       32.3   
80 0 days 01:20:54.437000     19.6      35.0    1016.0     False       32.3   
81 0 days 01:21:54.437000     19.6      36.0    1016.0     False       32.2   

    WindDirection  WindSpeed  
0               3   

TypeError: 'NoneType' object is not subscriptable

In [ ]:
import pandas as pd 
import fastf1 as f1 
from timple.timedelta import strftimedelta as td
# I am choosing the Australian GP randomly because it contains all the main drivers in their main team.  
F1_2022_AUS = f1.get_session(2022, "Australian Grand Prix", "R" )
F1_2022_AUS.load(telemetry=True, laps=True)
drivers_2022_list = F1_2022_AUS.results['Abbreviation'].tolist()

F1_2023_AUS = f1.get_session(2023, 'Australian Grand Prix', 'R')
F1_2023_AUS.load()
drivers_2023_list = F1_2023_AUS.results['Abbreviation'].tolist()
drivers_2023_list.append('RIC')


drivers_list_combined = list(set(drivers_2022_list+drivers_2023_list))
# we are gonna get the list of races for each year as well as 2024 
# we are gonna load in the list of name into the df 

# schedule for 2022 season 
schedule22 = f1.get_event_schedule(2022, include_testing=False, backend = 'ergast')
schedule_2022 = schedule22[ 'EventName'].tolist()

# schedule for 2023 season 
schedule23 = f1.get_event_schedule(2023, include_testing=False, backend = 'ergast')
schedule_2023 = schedule23['EventName'].tolist()

# schedule for 2024 season 
schedule24 = f1.get_event_schedule(2024, include_testing=False, backend = 'ergast')
schedule_2024 = schedule24['EventName'].tolist()



#get columns for data 
session_list_2022 = ['FP1', 'FP2','FP3','Q']
tyre_list = ['Soft','Medium', 'Hard', 'Intermediate','Wet']
#columns = []
columns_2022 = []
columns_2023 = [] 
for GP in schedule_2022:
  for session in session_list_2022:
        for tyre in tyre_list:
           if GP == 'Emilia Romagna Grand Prix' or  GP == 'Austrian Grand Prix' or GP == 'São Paulo Grand Prix': 
                if session == 'FP3': break
           else:
            columns_2022.append(GP+'_'+'2022'+'_'+ session +'_'+tyre) 


session_list_2023 = ['FP1', 'FP2','FP3','Q' ,'SS']
tyre_list = ['Soft','Medium', 'Hard', 'Intermediate','Wet']
sprint_weekend_2023 =['Azerbaijan Grand Prix', 'Austrian Grand Prix', 'São Paulo Grand Prix' , 'Belgian Grand Prix' ,  'Qatar Grand Prix', ' United States Grand Prix']
for GP in schedule_2023:
  for session in session_list_2023:
        if(GP not in sprint_weekend_2023)  and (session =='SS'):
            continue
        if(GP in sprint_weekend_2023) and (session == 'FP3' or session == 'FP2'): 
            continue  
        for tyre in tyre_list:
           columns_2023.append(GP+'_'+'2023'+'_'+ session +'_'+tyre) 

#df = pd.DataFrame(columns_2023)
#df.to_csv('columns2023_new.csv')

# so we will redo the get data function and use 1 for loop to pass in the columns data and then we will slice the string to get the data needed instead of having all of those parameters. 

# for GP that is passed in. we will run through a list of drivers and then get a list compiled and then we will 

def get_data(index):
      # this will split the string passed in and place them into a list. This is how we will get our data.
      # ["GP, year, session, tyre"]
      data = []
      #session_list = ['FP1', 'FP2','FP3','Q']
      #tyre_list = ['Soft','Medium', 'Hard']
      info = index.rsplit("_")
      session = f1.get_session(int(info[1]), info[0], info[2])
      session.load(laps=True, telemetry=False, weather=True, messages=False)  
      for driver in drivers_list_combined:
        
      

        """    print(get_weather(session))
        if get_weather(session) is not True and (info[3] == 'Intermediate' or info[3] == 'Wet'): 
           
            data = ['N/A'] * 24 
            print('here')
            break 
         """
        time = session.laps.pick_drivers(driver).pick_compounds(info[3]).pick_fastest()
       
        if time is not None and time['FastF1Generated'] == False and time['IsAccurate'] == True : 
                  print('First')
                  data.append( td(time['LapTime'], '%m:%s.%ms')) 
        else: 
                   print('Seccond')
                   data.append('N/A')
               
       
      print(data)
      return data   


    
# initialize list of lists
dataset = {'Name': drivers_list_combined}

 # Create the pandas DataFrame
df_2023 = pd.DataFrame(dataset)

for i in columns_2023:
   print(i)
   df_2023[i] = get_data(i)

df_2023.to_csv('File2023.csv')


 # Create the pandas DataFrame
df_2022= pd.DataFrame(dataset)


for i in columns_2022:
   print(i)
   
   df_2022[i] = get_data(i)
  
df_2022.to_csv('DataFile_2022_new.csv') 



"""  if time == None:
            data.append('N/A')
        else:
             if time['IsAccurate'] == True and time['FastF1Generated'] == False: 
               data.append(time['LapTime'])
             else:
                 data.append('N/A') """
"""  try:
            data.append(time['LapTime'])
        except TypeError :
            data.append('N/A') """ 
# Dutch Grand Prix, some problem with FP2 
# another problem is assigning the code too many times like the "df[i] = .... " so consider using df.concat and axis =1 or something 

core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 16 completed the race distance 00:00.140000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['16', '11', '63', '44', '4', '3', '31', '77', '10', '23', '24',

Bahrain Grand Prix_2023_FP1_Soft


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for weather_data
core           INFO 	Finished loading data for 20 drivers: ['1', '2', '4', '10', '11', '14', '16', '18', '20', '21', '22', '23', '24', '27', '31', '44', '55', '63', '77', '81']
core           INFO 	Loading data for Bahrain Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


Seccond
First
Seccond
Seccond
Seccond
Seccond
Seccond
First
First
First
First
First
First
First
Seccond
First
Seccond
First
Seccond
First
First
Seccond
Seccond
Seccond
['N/A', '01:34.997', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '01:33.375', '01:34.402', '01:34.575', '01:34.165', '01:35.043', '01:34.298', '01:35.015', 'N/A', '01:35.402', 'N/A', '01:32.758', 'N/A', '01:34.689', '01:33.196', 'N/A', 'N/A', 'N/A']
Bahrain Grand Prix_2023_FP1_Medium


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for weather_data
core           INFO 	Finished loading data for 20 drivers: ['1', '2', '4', '10', '11', '14', '16', '18', '20', '21', '22', '23', '24', '27', '31', '44', '55', '63', '77', '81']
core           INFO 	Loading data for Bahrain Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing ti

...

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for weather_data
core           INFO 	Finished loading data for 20 drivers: ['1', '2', '3', '4', '10', '11', '14', '16', '18', '20', '22', '23', '24', '27', '31', '44', '55', '63', '77', '81']
C:\Users\Phuc Trinh\AppData\Local\Temp\ipykernel_3520\57168882.py:110: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_2023[i] = get_data(i)
events      WARNING 	Correcting user input 'United States Grand Prix' to 'United States Grand Prix

Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
['N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
United States Grand Prix_2023_FP1_Wet


req            INFO 	Using cached data for weather_data
core           INFO 	Finished loading data for 20 drivers: ['1', '2', '3', '4', '10', '11', '14', '16', '18', '20', '22', '23', '24', '27', '31', '44', '55', '63', '77', '81']
C:\Users\Phuc Trinh\AppData\Local\Temp\ipykernel_3520\57168882.py:110: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_2023[i] = get_data(i)
events      WARNING 	Correcting user input 'United States Grand Prix' to 'United States Grand Prix'


Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
Seccond
['N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
United States Grand Prix_2023_FP2_Soft


ValueError: Session type 'FP2' does not exist for this event